# Section 4.3.1: Restored Notebook

This notebook was restored from a corrupted state. It likely contained an import statement for `japanize_matplotlib`.

In [ ]:
import japanize_matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd